<a href="https://colab.research.google.com/github/abhinavtk1/CS6910-A2/blob/main/main_part_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MA23M002 - ABHINAV T K
# CS6910 - Assignment 2

**Github repo:** https://github.com/abhinavtk1/CS6910-A2 <br>
**Wandb report:**

In [1]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Path: /content/drive/MyDrive/CS6910-A2
path = "/content/drive/MyDrive/CS6910-A2"


In [ ]:
# Install pytorch lightning
!pip install pytorch-lightning

In [ ]:
! pip install --quiet "pytorch-lightning>=1.4, <2.1.0" "urllib3" "ipython[notebook]>=8.0.0, <8.17.0" "torchmetrics>=0.7, <1.3" "torch>=1.8.1, <2.1.0" "matplotlib>=3.0.0, <3.9.0" "matplotlib" "lightning>=2.0.0" "setuptools>=68.0.0, <68.3.0"

# Part A: Training from scratch


Question 1

In [ ]:
IMG_SIZE = (128,128)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

In [2]:
# Reference: https://machinelearningmastery.com/building-a-convolutional-neural-network-in-pytorch/

class CNN(nn.Module):
    def __init__(self, num_filters = [32,32,64,64,128], kernel_size = [3]*5, act_fn = 'relu', num_neurons = 512):
        super().__init__()

        #Conv layer 1
        self.conv1 = nn.Conv2d(3, num_filters[0],
                               kernel_size=(kernel_size[0],kernel_size[0]),
                               stride=1, padding=1)
        if act_fn=='relu':
          self.act1 = nn.ReLU()
        elif act_fn=='tanh':
          self.act1 = nn.Tanh()
        elif act_fn=='sigmoid':
          self.act1 = nn.Sigmoid()
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))

        #Conv layer 2
        self.conv2 = nn.Conv2d(num_filters[0], num_filters[1],
                               kernel_size=(kernel_size[1],kernel_size[1]),
                               stride=1, padding=1)
        if act_fn=='relu':
          self.act2 = nn.ReLU()
        elif act_fn=='tanh':
          self.act2 = nn.Tanh()
        elif act_fn=='sigmoid':
          self.act2 = nn.Sigmoid()
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        #Conv layer 3
        self.conv3 = nn.Conv2d(num_filters[1], num_filters[2],
                               kernel_size=(kernel_size[2],kernel_size[2]),
                               stride=1, padding=1)
        if act_fn=='relu':
          self.act3 = nn.ReLU()
        elif act_fn=='tanh':
          self.act3 = nn.Tanh()
        elif act_fn=='sigmoid':
          self.act3 = nn.Sigmoid()
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2))

        #Conv layer 4
        self.conv4 = nn.Conv2d(num_filters[2], num_filters[3],
                               kernel_size=(kernel_size[3],kernel_size[3]),
                               stride=1, padding=1)
        if act_fn=='relu':
          self.act4 = nn.ReLU()
        elif act_fn=='tanh':
          self.act4 = nn.Tanh()
        elif act_fn=='sigmoid':
          self.act4 = nn.Sigmoid()
        self.pool4 = nn.MaxPool2d(kernel_size=(2, 2))

        #Conv layer 5
        self.conv5 = nn.Conv2d(num_filters[3], num_filters[4],
                               kernel_size=(kernel_size[4],kernel_size[4]),
                               stride=1, padding=1)
        if act_fn=='relu':
          self.act5 = nn.ReLU()
        elif act_fn=='tanh':
          self.act5 = nn.Tanh()
        elif act_fn=='sigmoid':
          self.act5 = nn.Sigmoid()
        self.pool5 = nn.MaxPool2d(kernel_size=(2, 2))

        # Flatten
        self.flatten = nn.Flatten()

        # dense layer
        self.fc6 = nn.Linear(2048, num_neurons)
        if act_fn=='relu':
          self.act6 = nn.ReLU()
        elif act_fn=='tanh':
          self.act6 = nn.Tanh()
        elif act_fn=='sigmoid':
          self.act6 = nn.Sigmoid()

        # output layer
        self.fc7 = nn.Linear(num_neurons, 10)


    def forward(self, x):
        # input 3x128x128, output 32x128x128
        x = self.act1(self.conv1(x))
        # input 32x128x128, output 32x64x64
        x = self.pool1(x)

        # input 32x64x64, output 32x64x64
        x = self.act2(self.conv2(x))
        # input 32x64x64, output 32x32x32
        x = self.pool2(x)

        # input 32x32x32, output 64x32x32
        x = self.act3(self.conv3(x))
        # input 64x32x32, output 64x16x16
        x = self.pool3(x)

        # input 64x16x16, output 64x16x16
        x = self.act4(self.conv4(x))
        # input 64x16x16, output 64x8x8
        x = self.pool4(x)

        # input 64x8x8, output 128x8x8
        x = self.act5(self.conv5(x))
        # input 128x8x8, output 128x4x4
        x = self.pool5(x)

        # input 128,4,4 output 2048
        x = self.flatten(x)

        # input 2048, output 512
        x = self.act6(self.fc6(x))

        # input 512, output 10
        x = self.fc7(x)

        return x